In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
#from dotenv import load_dotenv

#load_dotenv()  # take environment variables from .env.

In [ ]:
# os.getenv("HUGGINGFACE_API_KEY")
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ''

In [ ]:
#llm=HuggingFaceHub(repo_id="google/flan-t5-large",task="text-generation",model_kwargs={"temperature": 0.1})  

In [ ]:
#llm

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [ ]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [ ]:
quiz_chain=LLMChain(llm=HuggingFaceHub(repo_id="facebook/mbart-large-50",model_kwargs={
    "temperature": 0.6, "max_length":50}), prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [ ]:
review_chain=LLMChain(llm=HuggingFaceHub(repo_id="facebook/mbart-large-50",model_kwargs={
    "temperature": 0.6,"max_length":50}), prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
file_path="/home/phoenix/Downloads/mcqgenai/data.txt"

In [ ]:
file_path

In [ ]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

In [ ]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER=5 
SUBJECT="prompt engineering"
TONE="simple"


In [ ]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [ ]:
response

In [ ]:
quiz=response.get("quiz")
print(quiz)

In [ ]:
quiz=json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')